# Tutorial: run pyAFQ on data from infants

Infant brains are profoundly different from grownup brains.

## Get some baby diffusion data: 

The data for this example, provided by [Kalanit Grill Spector's Stanford Vision and Perception Neuroscience Lab](https://vpnl.stanford.edu/) is available to download [on Figshare](https://figshare.com/articles/dataset/Example_babyAFQ_BIDS_subject/21440739). You can download it from there and unzip it into `~/AFQ_Data/baby_example/` (Note that this is 2.69GB of data, so it can take a while to download). This data has been previously published in Grotheer et al. (2022).


In this case, a tractography has already been run with the excellent [MRtrix](https://www.mrtrix.org/) software. When you first run the following code, it will download the infant templates into your `~/AFQ_data/pediatric_templates` folder, organizing them there in the way that pyAFQ expects to find them.

In [ ]:
from AFQ.api.group import GroupAFQ
import AFQ.api.bundle_dict as abd
import AFQ.data.fetch as afd
import os.path as op
myafq = GroupAFQ(
    # ==== BIDS parameters ====
    bids_path=op.join(op.expanduser('~'), "AFQ_data/example_bids_subject/"),
    preproc_pipeline="vistasoft",
    # ===== Registration parameters ====
    reg_template_spec=afd.read_pediatric_templates()["UNCNeo-withCerebellum-for-babyAFQ"],
    reg_subject_spec="b0",
    # ==== Bundle parameters ====
    bundle_info=abd.PediatricBundleDict(),
    # ==== Tracking parameters ====
    import_tract={
        "suffix": "tractography", "scope": "mrtrix"},
    # ==== Segmentation parameters ====
    segmentation_params={
        "filter_by_endpoints": False},
    clean_params={
        'distance_threshold': 4},
)


### Step 3: Run pyAFQ
After defining your pyAFQ API object, you can ask for the output of any step of the pipeline. It is common for users to just call export_all (for example, myafq.export_all()).

In [ ]:
myafq.export("template_xform")

On the left, the babyAFQ template from MNI space has been transformed into the subject's native space. On the right, the subject's b0 is shown for reference.
![warped_template](warped_template.png)

In [ ]:
import plotly
bundle_html = myafq.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

### Another way to run this: Using the pyAFQ configuration file

pyAFQ provides a [command-line interface (CLI)](https://yeatmanlab.github.io/pyAFQ/usage/index.html). After installing the software, and organizing the data, run:

    pyAFQ /path/to/config.toml

At minimum, the file should contain the BIDS path:

    [BIDS_PARAMS]
    bids_path = "path/to/study"

But additional configuration options can be provided. Here is an example configuration file to run babyAFQ:

    [BIDS_PARAMS]
    bids_path = "/path/to/AFQ_data/example_bids_subject"
    preproc_pipeline = "vistasoft"

    [DATA]
    reg_template_spec = "pediatric"
    reg_subject_spec = "b0"
    bundle_info="PediatricBundleDict()"

    [TRACTOGRAPHY_PARAMS]
    import_tract = "{'suffix': 'tractography', 'scope': 'mrtrix'}"

    [SEGMENTATION_PARAMS]
    filter_by_endpoints = false
    
    [CLEANING_PARAMS]
    distance_threshold = 4

# References 

1. Grotheer M, Rosenke M, Wu H, Kular H, Querdasi FR, Natu VS, Yeatman JD, Grill-Spector K. White matter myelination during early infancy is linked to spatial gradients and myelin content at birth. Nat Commun. 2022 Feb 22;13(1):997. doi: 10.1038/s41467-022-28326-4. PMID: 35194018; PMCID: PMC8863985.